In [42]:
%load_ext autoreload
%autoreload 2
# load general packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from model_dc_ourmodel import model_dc_ourmodel
import time
import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
model = model_dc_ourmodel()
par   = model.par
sol   = model.sol

In [44]:
t0 = time.time()  # set the starting time
model.setup()
#tools.GaussHermite_lognorm(par.sigma_epsilon,par.Nxi)
model.create_grids()
model.solve()
t1 = time.time() # set the ending time
print(f'time: {t1-t0:.8} seconds') # print the total time

ValueError: setting an array element with a sequence.

In [ ]:
# Figure
def figure_multi_dim(par,sol,z,i_p):
    p = par.grid_p[i_p]
    if z == 0:
        print(f'Retire in t+1')
        ts = [par.T, par.T-1, par.T-2, par.T-3, par.T-4]
        print(f'p={p:.3}')
    elif z ==1:
        print(f'Working in t+1')
        print(f'p={p:.3}')
        ts = [par.T, par.T-1, par.T-2, par.T-3, par.T-4]
    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(1,1,1)
    for i in ts:
        ax.scatter(par.grid_m,sol.c[i-1,z,:,i_p], label=f't = {i}')
    ax.set_xlabel(f"$m_t$")
    ax.set_ylabel(f"$c(m_t,z_{{t+1}} = {z}, p = {p:.3})$")
    ax.set_xlim([0, 5])
    ax.set_ylim([0,3])
    ax.set_title(f'Consumption function')
    plt.legend()
    plt.show()

figure_multi_dim(model.par,model.sol,1,25)
figure_multi_dim(model.par,model.sol,1,50)
figure_multi_dim(model.par,model.sol,1,75)